In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import backend as K
import numpy as np
import os
import time
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session


config = tf.ConfigProto()
config.gpu_options.allow_growth = True  #dynamically grow the memory used on the GPU
set_session(tf.Session(config=config))

# dimensions of our images.
img_width, img_height = 512, 512

train_data_dir = 'E:\\UAV_drone\\data\\training\\vertical_view\\train'
validation_data_dir = 'E:\\UAV_drone\\data\\training\\vertical_view\\val'
nb_train_samples = 50041
nb_validation_samples = 12117
epochs = 100
batch_size = 16
img_channels = 3
nb_classes = 3

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(rescale=1. / 255)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(512, 512),
    batch_size=batch_size,
    classes=['down', 'medium', 'up'])

label_map = (train_generator.class_indices)
print(label_map)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(512, 512),
    batch_size=batch_size,
    classes=['down', 'medium', 'up'])


# checkpoint
current_directory = os.getcwd()

checkpoint_dir = os.path.join(current_directory, 'checkpoints')
result_dir = os.path.join(current_directory, 'results')

if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

Using TensorFlow backend.


Found 50041 images belonging to 3 classes.
{'down': 0, 'medium': 1, 'up': 2}
Found 12117 images belonging to 3 classes.


### Load base model

In [2]:
from keras.models import load_model    

model_file = 'model_densenet121_view-offset-altitude_11.hdf5'
model_full_path = os.path.join(result_dir, model_file)
model = load_model(model_full_path)

model.summary()

# weights = model.get_weights()
# print(len(weights))
# print(len(model.layers))
# print(len(weights[:-4]))
# for i in range(0, len(weights)):
#     print(weights[i].shape)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 518, 518, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 256, 256, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 256, 256, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

### Transfering to new model

In [3]:
from keras.models import Model
new_model = Model(model.inputs, model.layers[-3].output)

for i in range(0, (len(new_model.layers)-2)):
    new_model.layers[i].trainable = False
    
new_model.summary()
new_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

weights = model.get_weights()
new_model.set_weights(weights[:-4])

# delete(model)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 518, 518, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 256, 256, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 256, 256, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

conv4_block8_1_conv (Conv2D)    (None, 32, 32, 128)  61440       conv4_block8_0_relu[0][0]        
__________________________________________________________________________________________________
conv4_block8_1_bn (BatchNormali (None, 32, 32, 128)  512         conv4_block8_1_conv[0][0]        
__________________________________________________________________________________________________
conv4_block8_1_relu (Activation (None, 32, 32, 128)  0           conv4_block8_1_bn[0][0]          
__________________________________________________________________________________________________
conv4_block8_2_conv (Conv2D)    (None, 32, 32, 32)   36864       conv4_block8_1_relu[0][0]        
__________________________________________________________________________________________________
conv4_block8_concat (Concatenat (None, 32, 32, 512)  0           conv4_block7_concat[0][0]        
                                                                 conv4_block8_2_conv[0][0]        
__________

In [4]:
#check if weights are loaded into the new_model
new_model.get_weights()

[array([[[[-6.96834400e-02,  3.44931871e-01,  2.16513470e-01, ...,
           -6.52848482e-02, -3.01687509e-01,  3.48291606e-01],
          [ 1.65202379e-01,  4.74335074e-01, -2.05659699e-02, ...,
           -5.69609702e-01,  1.15547650e-01, -2.44396612e-01],
          [ 4.66973754e-03,  4.66952473e-02, -1.46678388e-01, ...,
            1.38492927e-01, -5.70146041e-03,  4.71694432e-02]],
 
         [[-7.45902732e-02,  4.60707307e-01,  7.35428408e-02, ...,
           -8.87244940e-02, -4.79835391e-01,  5.04609466e-01],
          [ 2.01543525e-01,  3.44618350e-01, -2.86626130e-01, ...,
           -6.87160909e-01, -7.15127289e-02,  8.33087787e-02],
          [-3.13685127e-02, -6.36311024e-02, -2.36395821e-01, ...,
           -5.92612326e-02,  1.55079095e-02,  5.45015752e-01]],
 
         [[-1.88848779e-01,  6.71832204e-01, -1.75598398e-01, ...,
           -1.18756341e-02, -4.40129727e-01,  3.75238001e-01],
          [ 2.11530209e-01,  4.57933962e-01, -3.71106476e-01, ...,
           -5.204

### Training the transfered-model

In [5]:
file="model_densenet121_cam_pitch_transfer-learning_{epoch:02d}.hdf5"
checkpoint =  ModelCheckpoint(os.path.join(checkpoint_dir, file), verbose=1)

NAME = "densenet121_cam-pitch_transfer-learning_{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME), write_graph = True, write_images=True)

new_model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[checkpoint, tensorboard])

Epoch 1/100
3127/3127 [==============================] - 1721s 550ms/step - loss: 1.1750 - acc: 0.7504 - val_loss: 0.6528 - val_acc: 0.8028

Epoch 00001: saving model to C:\Users\hoang\WorkingSpace\TrainingModels\keras\checkpoints\model_densenet121_cam_pitch_transfer-learning_01.hdf5
Epoch 2/100
3127/3127 [==============================] - 1742s 557ms/step - loss: 0.2176 - acc: 0.9396 - val_loss: 0.3692 - val_acc: 0.8703

Epoch 00002: saving model to C:\Users\hoang\WorkingSpace\TrainingModels\keras\checkpoints\model_densenet121_cam_pitch_transfer-learning_02.hdf5
Epoch 3/100
3127/3127 [==============================] - 1744s 558ms/step - loss: 0.1198 - acc: 0.9634 - val_loss: 0.2233 - val_acc: 0.9162

Epoch 00003: saving model to C:\Users\hoang\WorkingSpace\TrainingModels\keras\checkpoints\model_densenet121_cam_pitch_transfer-learning_03.hdf5
Epoch 4/100
3127/3127 [==============================] - 1725s 552ms/step - loss: 0.0911 - acc: 0.9725 - val_loss: 0.2266 - val_acc: 0.9200

Epoc

KeyboardInterrupt: 